In [12]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Documents/NFDI4Earth/juliaeoTerceira2023/handson_datacubes`


In this notebook you will learn how the chunking of the data set affects the reading and processing speed depending on the chunking and on the access patterns you need for your analysis.

# Understand chunking and rechunking

Task: Use your favorite NetCDF package and method to compute 

a) mean and 
b) median per spatial pixel

for the air_temperature_2m variable in this dataset without loading the whole data into memory (7GB uncompressed data per variable)

We first test access time along different axes:

In [13]:
using NetCDF
using DiskArrays: eachchunk

v = NetCDF.open("esdc_subset2_compressed.nc","layer")

Disk Array with size 1440 x 720 x 1794


In [14]:
@time v[:,:,1];

  0.003255 seconds (40 allocations: 3.956 MiB)


In [15]:
@time v[1,1,:];

 28.262194 seconds (38 allocations: 8.672 KiB)


In [16]:
vchunked = NetCDF.open("esdc_subset_compressed.nc","layer")

@time vchunked[:,:,1];

  2.529157 seconds (40 allocations: 3.956 MiB)


In [17]:
@time vchunked[1,1,:];

  0.240300 seconds (38 allocations: 8.672 KiB)


Access along spatial strides is much faster than access of time series
because of the internal storage in the netcdf file

In [18]:

eachchunk(v)

1×1×1794 DiskArrays.GridChunks{3}:
[:, :, 1] =
 (1:1440, 1:720, 1:1)

[:, :, 2] =
 (1:1440, 1:720, 2:2)

[:, :, 3] =
 (1:1440, 1:720, 3:3)

;;; … 

[:, :, 1792] =
 (1:1440, 1:720, 1792:1792)

[:, :, 1793] =
 (1:1440, 1:720, 1793:1793)

[:, :, 1794] =
 (1:1440, 1:720, 1794:1794)


DiskArrays.jl knows about the internal chunking structure and provides special implementations
for mapreduce which is used in the implementation of mean for AbstractArray
The following two aggregations access every chunk only once:


In [19]:

using Statistics
@time mean(v,dims=3)

@time mean(v,dims=(1,2))

 31.101223 seconds (4.04 M allocations: 7.130 GiB, 3.05% gc time, 2.98% compilation time)
 30.049550 seconds (707.62 k allocations: 6.962 GiB, 0.76% gc time, 0.56% compilation time)


1×1×1794 Array{Float32, 3}:
[:, :, 1] =
 NaN

[:, :, 2] =
 NaN

[:, :, 3] =
 NaN

;;; … 

[:, :, 1792] =
 277.7125

[:, :, 1793] =
 277.91547

[:, :, 1794] =
 277.78473


This gets more difficult for the median, because here we need the full ts in memory.
This makes it impossible to compute the median in a single pass
Let's try this on a small subset


In [49]:

v2 = view(v,1:2, 1:2,:)
out = zeros(size(v2,1),size(v2,2))

@time for ilat in axes(v2,2), ilon in axes(v2,1)
    out[ilon,ilat] = median(v2[ilon,ilat,:])
end

104.814163 seconds (2.51 M allocations: 139.902 MiB, 0.05% gc time, 0.56% compilation time)



This already takes ages to finish for 4 grid cells only. It would be better to e.g. always read approx 1GB of data at a time and consecutively do the computations:

In [20]:

using ProgressMeter
out = zeros(size(v,1),size(v,2))
latsteps = 90
latranges = [(i*90-latsteps+1):(i*90) for i in 1:(720 ÷ latsteps)]


8-element Vector{UnitRange{Int64}}:
 1:90
 91:180
 181:270
 271:360
 361:450
 451:540
 541:630
 631:720

In [51]:
@showprogress for ilat in latranges
    out[:,ilat] = median(v[:,ilat,:],dims=3)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:03:39




This finishes in a reasonable amount of time. 
Alternatively we can use YAXArrays.jl which does exactly this workflow for a given cache size


In [8]:
pwd()

"/home/fcremer/Documents/NFDI4Earth/juliaeoTerceira2023/handson_datacubes"

In [21]:
using YAXArrays
ds = open_dataset("esdc_subset_compressed.nc")

ds.layer

YAXArray with the following dimensions
lon                 Axis with 1440 Elements from -179.875 to 179.875
lat                 Axis with 720 Elements from 89.875 to -89.875
time                Axis with 1748 Elements from 1980-01-05T00:00:00 to 2017-12-31T00:00:00
name: layer
units: W m-2
Total size: 6.75 GB


In [55]:
medtair = mapslices(median, ds.layer, dims="Time", max_cache=1e9)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:16


YAXArray with the following dimensions
lon                 Axis with 1440 Elements from -179.875 to 179.875
lat                 Axis with 720 Elements from 89.875 to -89.875
Total size: 3.96 MB



YAXArrays will also take care of parallelization, for IO-limited processing tasks, we use Distributed.jl 

In [22]:
using Distributed, Zarr
addprocs(4)

4-element Vector{Int64}:
 6
 7
 8
 9

In [23]:
@everywhere begin 
    using Pkg
    Pkg.activate(".")
    Pkg.instantiate()
    #Pkg.status()
    using YAXArrays, Statistics, NetCDF, Zarr
end

  Activating project at `~/Documents/NFDI4Earth/juliaeoTerceira2023/handson_datacubes`


      From worker 4:	  Activating project at `~/Documents/NFDI4Earth/juliaeoTerceira2023/handson_datacubes`
      From worker 6:	  Activating project at `~/Documents/NFDI4Earth/juliaeoTerceira2023/handson_datacubes`
      From worker 7:	  Activating project at `~/Documents/NFDI4Earth/juliaeoTerceira2023/handson_datacubes`
      From worker 9:	  Activating project at `~/Documents/NFDI4Earth/juliaeoTerceira2023/handson_datacubes`
      From worker 8:	  Activating project at `~/Documents/NFDI4Earth/juliaeoTerceira2023/handson_datacubes`


In [24]:
medtair = mapslices(median, ds.layer, dims="Time", max_cache=1e9)

LoadError: ProcessExitedException(5)

In [25]:
rmprocs(workers())

Task (done) @0x00007f53945f0b90

# Avoiding "slow" data access by re-chunking

When repeatedly accessing data in an un-optimal way, rechunking might be an option. For example when you plan to develop some new method and you know that it will have to access the data from the time direction. 

In [26]:
dsrechunked = setchunks(ds,(time=184,lat=90,lon=90))

YAXArray Dataset
Dimensions: 
   lat                 Axis with 720 Elements from 89.875 to -89.875
   lon                 Axis with 1440 Elements from -179.875 to 179.875
   time                Axis with 1748 Elements from 1980-01-05T00:00:00 to 2017-12-31T00:00:00
Variables: layer 

In [27]:
savedataset(dsrechunked,path = "esdc_airtemp.zarr", overwrite=true)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:56


YAXArray Dataset
Dimensions: 
   lat                 Axis with 720 Elements from 89.875 to -89.875
   lon                 Axis with 1440 Elements from -179.875 to 179.875
   time                Axis with 1748 Elements from 1980-01-05T00:00:00 to 2017-12-31T00:00:00
Variables: layer 

Now we have created a new permanent copy of the dataset in zarr format and with relatively large chunks in time. This significantly speeds up computations along the time axis